<a href="https://colab.research.google.com/github/blowmeaway1234/Artificial-Intelligence/blob/main/Midterm_Recognition_Class_Member_build_with_ANN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Load dataset Class Member 

In [248]:
!ls gdrive/MyDrive/DataSetMember # test data 

testing.zip  trainingSet.zip


In [ ]:
# Unzip Test and Train data
! unzip gdrive/MyDrive/DataSetMember/testing.zip
! unzip gdrive/MyDrive/DataSetMember/trainingSet.zip

In [ ]:
# Check traindata
for i in range(10): # lệnh !ls không hiểu python nên ko dùng '{}'.format i
    !ls trainingSet/{i}| wc -l

In [251]:
# Check testdata
!ls testing/| wc -l

80


In [252]:
!ls trainingSet/| wc -l

10


# Preprocessing Data

In [253]:
import tensorflow as tf
from pathlib import Path

In [254]:
train_root = Path('trainingSet/')
testing_root=Path('testing/')
# convert to string type
filelist_ds= tf.data.Dataset.list_files(str(train_root/'*/*')) # 2 level scan (train_root to folders 0 - 9 then access each one)
test_list = tf.data.Dataset.list_files(str(testing_root/'*'))
# 9 random elements
for file in filelist_ds.take(8):
  print(file) #filename path in tf.data
for file in test_list.take(8):
  print(file) #filename path in tf.data

tf.Tensor(b'trainingSet/3/Long (7).jpg', shape=(), dtype=string)
tf.Tensor(b'trainingSet/9/Danh (56).jpg', shape=(), dtype=string)
tf.Tensor(b'trainingSet/1/Khiet (54).jpg', shape=(), dtype=string)
tf.Tensor(b'trainingSet/0/Hy (3).jpg', shape=(), dtype=string)
tf.Tensor(b'trainingSet/3/Long (37).jpg', shape=(), dtype=string)
tf.Tensor(b'trainingSet/8/Dien (20).jpg', shape=(), dtype=string)
tf.Tensor(b'trainingSet/0/Hy (35).jpg', shape=(), dtype=string)
tf.Tensor(b'trainingSet/7/Truc (3).jpg', shape=(), dtype=string)
tf.Tensor(b'testing/Truc (1).jpg', shape=(), dtype=string)
tf.Tensor(b'testing/Long (2).jpg', shape=(), dtype=string)
tf.Tensor(b'testing/Long (11).jpg', shape=(), dtype=string)
tf.Tensor(b'testing/Hy (25).jpg', shape=(), dtype=string)
tf.Tensor(b'testing/Hy (52).jpg', shape=(), dtype=string)
tf.Tensor(b'testing/Hy (57).jpg', shape=(), dtype=string)
tf.Tensor(b'testing/Truc (12).jpg', shape=(), dtype=string)
tf.Tensor(b'testing/Minh (28).jpg', shape=(), dtype=string)


In [255]:
def get_label(file_path):
  parts = tf.strings.split(file_path, '/')
  #part[0] = train
  #part[1] = 0/1.../9
  #part[2] = filename
  if parts[-2] == "0":
    labels=[1,0,0,0,0,0,0,0,0,0]
  elif parts[-2] == "1":
    labels=[0,1,0,0,0,0,0,0,0,0]
  elif parts[-2] == "2":
    labels=[0,0,1,0,0,0,0,0,0,0]
  elif parts[-2] == "3":
    labels=[0,0,0,1,0,0,0,0,0,0]
  elif parts[-2] == "4":
    labels=[0,0,0,0,1,0,0,0,0,0]
  elif parts[-2] == "5":
    labels=[0,0,0,0,0,1,0,0,0,0]
  elif parts[-2] == "6":
    labels=[0,0,0,0,0,0,1,0,0,0]
  elif parts[-2] == "7":
    labels=[0,0,0,0,0,0,0,1,0,0]
  elif parts[-2] == "8":
    labels=[0,0,0,0,0,0,0,0,1,0]
  else:
    labels=[0,0,0,0,0,0,0,0,0,1]
    #return labels value but have to convert to tensor becase string we use is python not tf.tensor
  return tf.convert_to_tensor(labels) 
# Test
for file in filelist_ds.take(8):
  print('filename',file.numpy().decode('utf-8')) # utf8 bỏ tf.tensor
  print('label:',get_label(file).numpy())

filename trainingSet/2/Kiet (52).jpg
label: [0 0 1 0 0 0 0 0 0 0]
filename trainingSet/0/Hy (1).jpg
label: [1 0 0 0 0 0 0 0 0 0]
filename trainingSet/5/Tan (11).jpg
label: [0 0 0 0 0 1 0 0 0 0]
filename trainingSet/6/Thang (56).jpg
label: [0 0 0 0 0 0 1 0 0 0]
filename trainingSet/4/Minh (28).jpg
label: [0 0 0 0 1 0 0 0 0 0]
filename trainingSet/3/Long (41).jpg
label: [0 0 0 1 0 0 0 0 0 0]
filename trainingSet/8/Dien (37).jpg
label: [0 0 0 0 0 0 0 0 1 0]
filename trainingSet/0/Hy (10).jpg
label: [1 0 0 0 0 0 0 0 0 0]


In [256]:
from tensorflow._api.v2 import image
# Preprocessing function
img_height = 30
img_width = 40
def preprocessing(file_path): #data we have just a file list but data for train model must be pixel value
  # Read file
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img,channels=3) # 1 is gray, 3 is color; we use image function of tf.image ; jpeg because jpg is our flie
  # Transform
    # Conver from uint 8 to float 32 and Normalize value to [0,1]
  img = tf.image.convert_image_dtype(img,tf.float32)
    # Resize
  img = tf.image.resize(img,[img_width,img_height])
    # Get image label
  label = get_label(file_path)
  # Return
  print(img.shape)
  return img, label

# Build Model

In [257]:
import tensorflow as tf
from tensorflow.keras import layers
#build model
model = tf.keras.Sequential()
inputs = tf.keras.Input(shape=(img_width,img_height,3))
x = layers.Conv2D(10,5,strides=(1,1),padding = 'valid',activation='relu')(inputs)
x = layers.MaxPool2D(pool_size=(2,2),strides=2)(x)
x = layers.Conv2D(20,5,strides=(1,1),padding='valid',activation='relu')(x)
x = layers.MaxPool2D(pool_size=(2,2),strides=2)(x)
x = layers.Dropout(0.25)(x)
x = layers.Flatten()(x)
x = layers.Dense(100,activation='relu')(x)
output = layers.Dense(10, activation ='softmax')(x)

model = tf.keras.Model(inputs=inputs,outputs=output)
model.summary()

Model: "model_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_39 (InputLayer)       [(None, 40, 30, 3)]       0         
                                                                 
 conv2d_48 (Conv2D)          (None, 36, 26, 10)        760       
                                                                 
 max_pooling2d_46 (MaxPoolin  (None, 18, 13, 10)       0         
 g2D)                                                            
                                                                 
 conv2d_49 (Conv2D)          (None, 14, 9, 20)         5020      
                                                                 
 max_pooling2d_47 (MaxPoolin  (None, 7, 4, 20)         0         
 g2D)                                                            
                                                                 
 dropout_23 (Dropout)        (None, 7, 4, 20)          0  

In [258]:
ds_size = 25000 #(shuffle size) 
# Set parameter
batch_size = 16
# Build data
train_ds = filelist_ds.shuffle(ds_size) # Shuffling the input and it doesn't have labels
train_ds = train_ds.map(preprocessing,num_parallel_calls=tf.data.AUTOTUNE) # Process each element + get labels => train_ds = pixel val and labels
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(8) # chia luồn data

(40, 30, 3)


# Compile and training model

In [259]:
# Compile model
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics=["accuracy"] #caculate accuracy on each batch
)

In [260]:
# Train model
model.fit(train_ds, epochs = 10)

Epoch 1/10
38/38 [==============================] - 2s 9ms/step - loss: 1.6548 - accuracy: 0.4750
Epoch 2/10
38/38 [==============================] - 0s 7ms/step - loss: 0.3753 - accuracy: 0.9000
Epoch 3/10
38/38 [==============================] - 0s 11ms/step - loss: 0.1676 - accuracy: 0.9517
Epoch 4/10
38/38 [==============================] - 0s 10ms/step - loss: 0.1361 - accuracy: 0.9600
Epoch 5/10
38/38 [==============================] - 0s 11ms/step - loss: 0.0707 - accuracy: 0.9850
Epoch 6/10
38/38 [==============================] - 0s 12ms/step - loss: 0.0708 - accuracy: 0.9833
Epoch 7/10
38/38 [==============================] - 0s 9ms/step - loss: 0.0468 - accuracy: 0.9883
Epoch 8/10
38/38 [==============================] - 0s 7ms/step - loss: 0.0495 - accuracy: 0.9850
Epoch 9/10
38/38 [==============================] - 0s 7ms/step - loss: 0.0251 - accuracy: 0.9950
Epoch 10/10
38/38 [==============================] - 0s 7ms/step - loss: 0.0217 - accuracy: 0.9967


In [261]:
model.save('ClassMemberANNmodel.h5')

In [ ]:
import matplotlib.pyplot as plt
from keras.utils import load_img
from keras.utils.image_utils import img_to_array
import numpy as np

name = {0: 'Hy', 1: 'Khiết', 2: 'Kiệt', 3: 'Long', 4: 'Minh', 5: 'Tấn', 6: 'Thắng', 7: 'Trúc', 8: 'Điển', 9: 'Danh'}
# Load 10 picture to scan the image
for i in range(9):
    img = load_img("Danh{i}.jpg".format(i=i), target_size=(40, 30))
    plt.imshow(img)
    img = img_to_array(img)
    img = img.reshape(1, 40, 30, 3)
    img = img.astype('float32')
    img = img / 255
    result = np.argmax(model.predict(img), axis=1)
    plt.show()
    print(name[result[0]])
